In [1]:
import tensorflow as tf


In [28]:
"""加载数据集合"""
# 定义数据集合的文件路径
# D 盘
train_dir = 'D:/Paper reproduction/RiceNet A deep convolutional neural network approach for classification of rice varieties/Dataset/items_train_val_set/train'
valid_dir = 'D:/Paper reproduction/RiceNet A deep convolutional neural network approach for classification of rice varieties/Dataset/items_train_val_set/validation'
dvi_valid_dir = 'D:/Paper reproduction/RiceNet A deep convolutional neural network approach for classification of rice varieties/Dataset/items_val_test_set/valid'
dvi_test_dir = 'D:/Paper reproduction/RiceNet A deep convolutional neural network approach for classification of rice varieties/Dataset/items_val_test_set/test'

'''
# C 盘
train_dir = 'C:/Users/86155/Desktop/Research/Deep Learning/Image classification application/RiceNet A deep convolutional neural network approach for classification of rice varieties/code_recurrence/Dataset/items_train_val_set/train'
valid_dir = 'C:/Users/86155/Desktop/Research/Deep Learning/Image classification application/RiceNet A deep convolutional neural network approach for classification of rice varieties/code_recurrence/Dataset/items_train_val_set/validation'
dvi_valid_dir = 'C:/Users/86155/Desktop/Research/Deep Learning/Image classification application/RiceNet A deep convolutional neural network approach for classification of rice varieties/code_recurrence/Dataset/items_val_test_set/valid'
dvi_test_dir = 'C:/Users/86155/Desktop/Research/Deep Learning/Image classification application/RiceNet A deep convolutional neural network approach for classification of rice varieties/code_recurrence/Dataset/items_val_test_set/test'
'''
# 定义批次大小
batch_size = 32
# 定义图像大小
image_size = (512,512)

# 配置训练数据集合的数据增强
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,         # 图像进行归一化处理
    rotation_range=20,      # 旋转角范围 0~20
    width_shift_range=0.2,  # 水平随机移动范围
    height_shift_range=0.2, # 垂直随机移动范围
    shear_range=0.2,        # 剪切强度
    zoom_range=0.2,         # 随机缩放的范围
    horizontal_flip=True    # 是否随机水平翻转
)
train_generator = train_datagen.flow_from_directory(
        train_dir,                # train 数据集合 的文件路径
        target_size=image_size,   # 输出的图像的大小
        batch_size=batch_size,    # 输出的批次大小
        class_mode='categorical') # 选取分类标签的形式 ， 这里是独热编码 (因为loss是交叉熵损失函数 categorical_crossentropy)


valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory(
        dvi_valid_dir,                # train 数据集合 的文件路径
        target_size=image_size,   # 输出的图像的大小
        batch_size=batch_size,    # 输出的批次大小
        class_mode='categorical') # 选取分类标签的形式 ， 这里是独热编码 (因为loss是交叉熵损失函数 categorical_crossentropy)

# 查看标签分配的类别对应的索引
train_generator.class_indices , valid_generator.class_indices


FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'D:/Paper reproduction/RiceNet A deep convolutional neural network approach for classification of rice varieties/Dataset/items_train_val_set/train'

In [23]:
import os
"""找出一共需要多少次批次 tatal_sample = batch_size * num_steps"""

total_class = os.listdir(train_dir)
print('共有 {} 个类别 , 分别是 ： {}'.format(len(total_class),total_class))

total_train_sample_number = 0
for signel_class in total_class:
    path = os.path.join(train_dir,signel_class)
    total_train_sample_number += len(os.listdir(path))
print('训练样本的总数：{}'.format(total_train_sample_number))

total_validation_sample_number = 0
for signel_class in total_class:
    path = os.path.join(valid_dir,signel_class)
    total_validation_sample_number += len(os.listdir(path))
print('测试集合和验证集合样本的总数：{}'.format(total_validation_sample_number))

total_valid_sample_number = 0
for signel_class in total_class:
    path = os.path.join(dvi_valid_dir,signel_class)
    total_valid_sample_number += len(os.listdir(path))
print('验证集合样本的总数：{}'.format(total_valid_sample_number))

total_test_sample_number = 0
for signel_class in total_class:
    path = os.path.join(dvi_test_dir,signel_class)
    total_test_sample_number += len(os.listdir(path))
print('测试集合样本的总数：{}'.format(total_test_sample_number))


共有 5 个类别 , 分别是 ： ['jehlum', 'mushkbudji', 'sr-1', 'sr-2', 'sr-4']
训练样本的总数：3430
测试集合和验证集合样本的总数：1470
验证集合样本的总数：735
测试集合样本的总数：735


In [5]:
# 定义模型
def RiceNet_Block(Conv2D_filters, Conv2D_kernel_size, Conv2D_strides, Conv2D_padding, Activation = 'relu'):
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters=Conv2D_filters, kernel_size = Conv2D_kernel_size, strides = Conv2D_strides, padding = Conv2D_padding, use_bias=True),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation(Activation)
    ])

def RiceNet():
    RiceNet = tf.keras.Sequential([
        # Input layer
        tf.keras.layers.InputLayer(input_shape = (512,512,3)),

        # big block 1
        RiceNet_Block(Conv2D_filters = 16, Conv2D_kernel_size = (5,5), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        # big block 2
        RiceNet_Block(Conv2D_filters = 16, Conv2D_kernel_size = (5,5), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
        # big block 3
        RiceNet_Block(Conv2D_filters = 32, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
        # big block 4
        RiceNet_Block(Conv2D_filters = 32, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
        # big block 5
        RiceNet_Block(Conv2D_filters = 64, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
        # big block 6
        RiceNet_Block(Conv2D_filters = 64, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
         # big block 7
        RiceNet_Block(Conv2D_filters = 128, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
        # big block 8
        RiceNet_Block(Conv2D_filters = 128, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
        # big block 9
        RiceNet_Block(Conv2D_filters = 192, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
        # big block 10
        RiceNet_Block(Conv2D_filters = 192, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),

        # FC layer
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units = 192),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.5),

        # Output layer
        tf.keras.layers.Dense(units = 5),
        tf.keras.layers.Softmax()
    ])

    return RiceNet

RiceNet = RiceNet()
# 查看模型结构
RiceNet.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 512, 512, 16)      1280      
                                                                 
 sequential_1 (Sequential)   (None, 512, 512, 16)      6480      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 256, 16)     0         
 )                                                               
                                                                 
 sequential_2 (Sequential)   (None, 256, 256, 32)      4768      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 128, 128, 32)     0         
 2D)                                                             
                                                                 
 sequential_3 (Sequential)   (None, 128, 128, 32)    

In [6]:
# 训练
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
'''
checkpoint_path = "training_8/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,monitor='val_acc', save_best_only=True, mode='auto',
                                                 verbose=1)
'''
lr = 0.00003
num_epochs = 50
# 直接返回 steps 的方法
# total_train_samples = len(train_generator.filenames)
# total_validation_samples = len(test_generator.filenames)
# 配置模型
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
RiceNet.compile(optimizer=optimizer,loss=loss,metrics = ['accuracy'])
# 训练模型
RiceNet.fit(train_generator,epochs = num_epochs, verbose = 2, steps_per_epoch = total_train_sample_number // batch_size, validation_data = valid_generator,validation_steps = total_valid_sample_number // batch_size)


Epoch 1/50
107/107 - 129s - loss: 1.6271 - accuracy: 0.3808 - val_loss: 1.8268 - val_accuracy: 0.2003 - 129s/epoch - 1s/step
Epoch 2/50
107/107 - 165s - loss: 1.1446 - accuracy: 0.5556 - val_loss: 2.1232 - val_accuracy: 0.2003 - 165s/epoch - 2s/step
Epoch 3/50
107/107 - 171s - loss: 0.9583 - accuracy: 0.6183 - val_loss: 2.3090 - val_accuracy: 0.2003 - 171s/epoch - 2s/step
Epoch 4/50
107/107 - 172s - loss: 0.8655 - accuracy: 0.6489 - val_loss: 2.3945 - val_accuracy: 0.1974 - 172s/epoch - 2s/step
Epoch 5/50
107/107 - 170s - loss: 0.7562 - accuracy: 0.6957 - val_loss: 2.2610 - val_accuracy: 0.3920 - 170s/epoch - 2s/step
Epoch 6/50
107/107 - 169s - loss: 0.7150 - accuracy: 0.7172 - val_loss: 1.2925 - val_accuracy: 0.6207 - 169s/epoch - 2s/step
Epoch 7/50
107/107 - 172s - loss: 0.6301 - accuracy: 0.7546 - val_loss: 0.4710 - val_accuracy: 0.8068 - 172s/epoch - 2s/step
Epoch 8/50
107/107 - 170s - loss: 0.5934 - accuracy: 0.7678 - val_loss: 0.4553 - val_accuracy: 0.8068 - 170s/epoch - 2s/step


KeyboardInterrupt: 

In [26]:
# transfer model of  InceptionV3

# load pre-training model

pre_training_InceptionV3 = tf.keras.applications.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(512, 512, 3),
    pooling=None,
)
lr = 0.00003
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = lr)

# The first steps

# 参数冻结
pre_training_InceptionV3.trainable = False
layer_229_output = pre_training_InceptionV3.output
x = tf.keras.layers.GlobalAveragePooling2D()(layer_229_output)
x = tf.keras.layers.Dense(units = 1024)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(units = 5)(x)
x = tf.keras.layers.Softmax()(x)

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

first_step_InceptionV3 = tf.keras.Model(inputs = pre_training_InceptionV3.input, outputs = x)

first_step_InceptionV3.compile(optimizer=optimizer,loss=loss,metrics = ['accuracy'])
first_step_InceptionV3.fit(train_generator,epochs = 30, verbose = 2, steps_per_epoch = total_train_sample_number // batch_size, validation_data = valid_generator,validation_steps = total_valid_sample_number // batch_size)


# The second steps
idx_mixed7 = 0
for i,layer in enumerate(pre_training_InceptionV3.layers):
    if layer.name == 'mixed7':
        idx_mixed7 = i

for layer in first_step_InceptionV3.layers[idx_mixed7+1:]:
     layer.trainable = True

InceptionV3 =  tf.keras.Model(inputs = first_step_InceptionV3.input, outputs = first_step_InceptionV3.output)
InceptionV3.compile(optimizer=optimizer,loss=loss,metrics = ['accuracy'])
InceptionV3.fit(train_generator,epochs = 30, verbose = 2, steps_per_epoch = total_train_sample_number // batch_size, validation_data = valid_generator,validation_steps = total_valid_sample_number // batch_size)


107/107 - 156s - loss: 1.1471 - accuracy: 0.5447 - val_loss: 1.3294 - val_accuracy: 0.6847 - 156s/epoch - 1s/step
107/107 - 186s - loss: 0.5111 - accuracy: 0.8011 - val_loss: 0.4217 - val_accuracy: 0.9304 - 186s/epoch - 2s/step


In [27]:
# transfer model of InceptionResNetV2

# load pre-training model
pre_training_InceptionResNetV2 = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
    include_top = False,
    weights = 'imagenet',
    input_shape = (512,512,3),
    pooling = None,
)

lr = 0.00003
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = lr)

# The first steps
pre_training_InceptionResNetV2.trainable = False
layer_InceptionResNetV2_output = pre_training_InceptionResNetV2.output
x = tf.keras.layers.GlobalAveragePooling2D()(layer_InceptionResNetV2_output)
X = tf.keras.layers.Dense(units = 512)(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Activation('relu')(x)
X = tf.keras.layers.Dense(units = 128)(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dense(units = 5)(x)
x = tf.keras.layers.Softmax()(x)
first_step_InceptionResNetV2 = tf.keras.Model(inputs = pre_training_InceptionResNetV2.input, outputs = x)

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

first_step_InceptionResNetV2.compile(optimizer=optimizer,loss=loss,metrics = ['accuracy'])
first_step_InceptionResNetV2.fit(train_generator,epochs = 30, verbose = 2, steps_per_epoch = total_train_sample_number // batch_size, validation_data = valid_generator,validation_steps = total_valid_sample_number // batch_size)

# The second steps
first_step_InceptionResNetV2.trainable = True
InceptionResNetV2 = tf.keras.Model(inputs = first_step_InceptionResNetV2.input, outputs = first_step_InceptionResNetV2.output)

InceptionResNetV2.compile(optimizer=optimizer,loss=loss,metrics = ['accuracy'])
InceptionResNetV2.fit(train_generator,epochs = 30, verbose = 2, steps_per_epoch = total_train_sample_number // batch_size, validation_data = valid_generator,validation_steps = total_valid_sample_number // batch_size)


   221184/219055592 [..............................] - ETA: 28:14

KeyboardInterrupt: 